## 0. Basic setup (library imports, database connection, function setup)

In [ ]:
# libraries
import psycopg2 as pg
import pandas as pd
import os

# set environment variable for psycog2 (for some systems)
os.environ["PGGSSENCMODE"] = "disable"

# get the stored passwords
f=open("credentials_david.txt", "rt")
pwd=f.readline().strip()  #di   password
hpwd=f.readline().strip() #home password
f.close()

In [ ]:
# di test: WORKING
#host_di = "appserver-01.alunos.di.fc.ul.pt"
#db_di = "tpd012"
#user_di = "tpd012"

#conn = pg.connect(host = host_di,
#                  database = db_di,
#                  user = user_di,
#                  password = pwd)
#conn.close()

# local test: WORKING
host_local = "localhost"
db_local = "tpd012"
user_local = "postgres"

conn = pg.connect(host = host_local,
                  database = db_local,
                  user = user_local,
                  password=hpwd)
conn.close()

In [ ]:
# functions
def excuteSingleSQLstatement(sql, host, database, user, password):
    conn = pg.connect(host = host,
                      database = database,
                      user = user,
                      password = password)
    cur = conn.cursor()
    cur.execute(sql)
    cur.close()
    conn.commit()
    conn.close() 

## 1. Creating _Property_ dimension table in SQL

